In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import seaborn as sns
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from sklearn.metrics import r2_score, accuracy_score, f1_score, confusion_matrix, fbeta_score, confusion_matrix

In [ ]:
#Load data
df_tweets = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/tweets.pkl', 'rb'))
df_bitcoin = pkl.load(open("/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/bitcoins.pkl", 'rb'))
df_googletrends = pkl.load(open("/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/Google_trends/google_trends.pkl", 'rb'))

In [ ]:
df_bitcoin.head()

In [ ]:
df_bitcoin1 = df_bitcoin.copy()
df_bitcoin1['Day'] = df_bitcoin1['Date'].apply(lambda x: pd.Timestamp(year =x.year, month = x.month, day = x.day, hour = 0, minute = 0, second = 0))
df_bitcoin1.drop_duplicates(subset = 'Day', inplace = True)

In [ ]:
df_svi = df_googletrends[['Day', 'SVI']]
df_svi = df_svi.merge(df_bitcoin1[['Day', 'Close']], on = 'Day', how = 'left')

In [ ]:
df_svi.to_csv('SVI_Price.csv')

In [ ]:
df_bitcoin[['Date', 'Close']].to_csv('Bitcoin_prices.csv')

In [ ]:
df_tweets.columns

In [ ]:
pos_neg_df = df_tweets[['Day','pos_neg_ratio_month']]

In [ ]:
pos_neg_df = pos_neg_df.drop_duplicates(subset = 'Day')

In [ ]:
topics_df['Day'] = topics_df.index

In [ ]:
pos_neg_df.to_csv('Pos_neg_ratio_month.csv')

In [ ]:
#Save Data for "Topic over Time" graphs as csv 
topicovertime = df_tweets.groupby(['Day', 'Topic'])['tweet-id'].count().reset_index()
topics_df = pd.DataFrame(columns = topicovertime['Topic'].unique(), index = df_tweets['Day'].unique())
for topic in topics_df.columns:
    value=topicovertime[topicovertime.Topic == topic]['tweet-id'].values
    topics_df[topic] = value
    topics_df.reset_index()

#topics_df.to_csv('Topic_time_graph.csv')

In [ ]:
df_csv = topics_df.merge(pos_neg_df, on = 'Day', how = 'left', )

In [ ]:
df_csv.to_csv('topic_time_graph.csv')

In [ ]:
topicovertime.head()

In [ ]:
topicovertime[topicovertime.Topic == 'Blockchain News']

In [ ]:
df_tweets_model=df_tweets[['hours','Topic', 'sentiment_compound',
       'Blockchain_News_proba', 'Cryptocurrency_proba', 'Market_News_proba',
       'sentiment', 'topic_activity_hour',
       'sentiment_topic_count_hour', 'topic_activity_day',
       'sentiment_topic_count_day', 'Negative_this_Day', 'Neutral_this_Day',
       'Positive_this_Day', 'Negative_this_Month', 'Neutral_this_Month',
       'Positive_this_Month', 'pos_neg_ratio_day', 'pos_neg_ratio_month',
       'Negative_this_hours', 'Neutral_this_hours', 'Positive_this_hours',
       'pos_neg_ratio_hour']]

In [ ]:
df_tweets_model = df_tweets_model.rename(columns = {'hours':'Date'})

In [ ]:
df_tweets_model.Date.head()

In [ ]:
df_tweets_model.drop_duplicates(subset = 'Date', inplace = True)

In [ ]:
df_tweets_model.head()

In [ ]:
df_model = df_tweets_model.merge(df_bitcoin, on = 'Date', how = 'left')

In [ ]:
df_model.columns

In [ ]:
df_model.insert(1, 'Day', df_model['Date'])

In [ ]:
df_model['Day'] = df_model['Date'].apply(lambda x: pd.Timestamp(year =x.year, month = x.month, day = x.day, hour = 0, minute = 0, second = 0))
df_googletrends['Day'] = df_googletrends['date'].apply(lambda x: pd.Timestamp(year =x.year, month = x.month, day = x.day, hour = 0, minute = 0, second = 0))

In [ ]:
df_model1= df_model.merge(df_googletrends, on = 'Day', how = 'left')

In [ ]:
df_model1.drop('date', axis = 1, inplace = True)
pkl.dump(df_model1, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/Model,pkl', 'wb'))

In [ ]:
df = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/Model,pkl', 'rb'))

In [ ]:
df[['Day','Close']].to_csv('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/Prices_dates.csv')

In [ ]:
sorted(df.Date.unique())

In [ ]:
#Split into train test set
def splitdata(df,target, test_size = 0.15, seed = 89, plot = False , deletefuture = False, classify = False, dummy = True, profit = False):
    """
    Split data into train and validation set according to day.

    Parameter df: Dataframe to be split
    Precondtion: df is a Pandas DataFrame

    Parameter test_size: Size of test data set
    Precondition: 0 ≤ test_size ≤ 1

    Parameter seed: random number for random state generator
    Precondition: type(seed) == int

    Parameter identifier: Dataset feature according to which the dataset will be split
    Precondition: identifier is a valid DataFrame index
    """
    days = df['Day']
    target_values = df[target]
    if classify:
        #Convert prices into 1(price increased) and 0 (price decreased)
        df[target] = df[target].apply(lambda x: convertprices(x))
    if deletefuture:
        #Delete all future prices from the datafram
        df = df.drop(labels = ['Date', 'Day', 'price_1hour_ahead', 'price_2days_ahead','price_1week_ahead',
       'price_2weeks_ahead', 'price_1month_ahead'] , axis = 1)
    if dummy and deletefuture:
        #Create dummies
        df = pd.get_dummies(df,drop_first = True)
        df['Day'] = days
    elif dummy:
        df = pd.get_dummies(df, drop_first = True)
        df['Day'] = days
    #rs = np.random.RandomState(seed)
    members_unique = df.Day.unique()
    members_unique = np.sort(members_unique)
    length = len(members_unique)
    cutoff = int(length * (1-test_size))
    test_members= members_unique[cutoff:]
    #test_members = rs.choice(members_unique, size = int(members_unique.shape[0]*\
    #test_size), replace = False)
    if profit:
        df_profit = df.copy()
        df_profit['target'] = target_values
        df_profit_tr = df_profit[~df_profit['Day'].isin(test_members)]
        df_profit_te = df_profit[df_profit['Day'].isin(test_members)]
    df_tr = df[~df['Day'].isin(test_members)]
    df_te = df[df['Day'].isin(test_members)]
    if classify:
        df_tr.drop(labels = 'Day', inplace = True, axis = 1)
        df_te.drop(labels = 'Day', inplace = True, axis = 1)
    
    if plot:
        sns.pairplot(df_tr)

    y_tr, y_te = df_tr[target], df_te[target]
    X_tr = df_tr.drop(columns = target, axis = 1)
    X_te = df_te.drop(columns = target, axis = 1)

    if profit:
        return X_tr, X_te, y_tr, y_te, df_profit_tr, df_profit_te

    return X_tr, X_te, y_tr, y_te

In [ ]:
def convertprices(price):
    #Convert prices into 1(price increased) and 0 (price decreased)
    if price >= 0:
        return 1
    if price < 0:
        return 0

In [ ]:
X_tr, X_te, y_tr, y_te = splitdata(df, 'price_1day_ahead', dummy = True)

In [ ]:
X_tr.head()

In [ ]:
y_tr[y_tr < 0]

In [ ]:
X_tr.Day.unique()

In [ ]:
#Dump final test set
pkl.dump(X_te, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/X_test', 'wb'))
pkl.dump(y_te, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/y_test', 'wb'))

In [ ]:
X_te['price_1month_ahead'] = y_te
testdf = X_te.copy()
y_test = y_te.copy()

In [ ]:
X_tr['price_1month_ahead'] = y_tr
X_tr, X_te, y_tr, y_te, df_profit_tr, df_profit_te = splitdata(df, 'price_1month_ahead', deletefuture = True, classify = True, profit = True, dummy = True)

In [ ]:
#Try model with regression using the actual price differences to predict the exact price increase
#Convert all columns to 1 and 0

### Classification Model

In [ ]:
#Fit KNN regression to training data and find optimal no. of neighbors
def fitknn(X_train, X_test, y_train ,y_test):
    d_knn = defaultdict(int)
    score = 0
    k = 0
    for i in range(1,50):
        neigh = KNeighborsClassifier(n_neighbors=i)
        neigh.fit(X_train, y_train)
        preds = neigh.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        d_knn[str(i)] = accuracy
        if accuracy > score:
            score = accuracy
            k = i
    print('k value: '+str(k)+'\n' 
          'accuracy: '+str(score))
    return d_knn

In [ ]:
#Fit Logistics regression to training data and find optimal 'C' value
def fitlog(X_train, X_test, y_train ,y_test):
    #Create dictionary to store values
    d_logistic = defaultdict(int)
    score = 0
    k = 1
    model = 0
    for i in range(1,1002,100):
        logistic = LogisticRegression(C = i)
        logistic.fit(X_train, y_train)
        preds = logistic.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        d_logistic[str(i)] = accuracy
        if accuracy > score:
            score = accuracy
            k = i 
            model = logistic
    print('C value: '+str(k)+'\n' 
          'accuracy: '+str(score))
    return d_logistic, model

In [ ]:
#Fit Decision Tree Classifier to training data
def fitrf(X_train, X_test, y_train ,y_test):
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    print('Random Forest Classifier accuracy: '+str(accuracy))
    return rf

In [ ]:
def xgboost(X_train, X_test, y_train, y_test):
    gbm = xgb.XGBClassifier(n_estimators=30000,
                            max_depth=5,
                            objective='binary:logistic',
                            learning_rate=.05, 
                            subsample=.8,
                            min_child_weight=2,
                            colsample_bytree=.8)

    eval_set=[(X_train,y_train),(X_test,y_test)]
    fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='error', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=40,
                    verbose=False
                   )

    print(accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit))) 
    return gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)

In [ ]:
def lightgbm(X_train, X_test, y_train, y_test, categorical = None):
    lgb_train = lgb.Dataset(X_train, y_train, feature_name='auto',categorical_feature=categorical)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, feature_name='auto', categorical_feature=categorical)    
    
    params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 5,
    'max_depth': 5,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    }
    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10000,
                valid_sets=lgb_eval,early_stopping_rounds=50)
    y_preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    y_preds1 = gbm.predict(X_train,num_iteration = gbm.best_iteration)
    
    
    df_imp = pd.DataFrame({'feature' : gbm.feature_name(),
                      'importance' : gbm.feature_importance(importance_type='gain')})
    return y_preds, y_preds1, gbm, df_imp

In [ ]:
def accuracy(actuals, preds):
    return np.mean(actuals == preds)

def precision(actuals,preds):
    tp = np.sum((actuals == 1) & (preds == 1))
    fp = np.sum((actuals == 0) & (preds == 1))
    return tp / (tp + fp)

def recall(actuals, preds):
    tp = np.sum((actuals == 1) & (preds == 1))
    fn = np.sum((actuals == 1) & (preds == 0))
    return tp / (tp + fn)

def F1(actuals, preds):
    p, r = precision(actuals, preds), recall(actuals, preds)
    return 2*p*r / (p + r)

def f1_beta(actuals, preds, beta):
    return fbeta_score(actuals, preds, beta)

In [ ]:
#Fit KNN Classifier
d_knn = fitknn(X_tr, X_te, y_tr, y_te)

In [ ]:
#Fit a logistic regression
d_logistic, model = fitlog(X_tr, X_te, y_tr, y_te)

In [ ]:
rf = fitrf(X_tr, X_te, y_tr, y_te)

In [ ]:
y_predict = xgboost(X_tr, X_te, y_tr, y_te)

In [ ]:
X_tr.head()

In [ ]:
y_preds_val, y_preds_tr, gbm, df_imp = lightgbm(X_tr, X_te, y_tr, y_te)

In [ ]:
X_test = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/X_test', 'rb'))
y_test = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/y_test', 'rb'))

In [ ]:
X_test

In [ ]:
#predictions_test = gbm.predict(testdf.drop(['Day', 'Date', 'price_1month_ahead'], axis = 1), num_iteration=gbm.best_iteration)
predictions = gbm.predict(testdf.drop(['Day', 'Date', 'price_1month_ahead'], axis = 1), num_iteration=gbm.best_iteration)

In [ ]:
y_preds_val

In [ ]:
y_tr[y_tr == 0]

In [ ]:
y_preds_val_hard = y_preds_val > 0.9
y_preds_tr_hard = y_preds_tr > 0.75
predictions_hard = predictions > 0.5

In [ ]:
X_te['predictions'] = y_preds_val_hard
X_te['predictions_soft'] = y_preds_val
X_te['predictions'].value_counts()
testdf['predictions'] = predictions_hard
testdf['profits'] = (y_test/100)*testdf['Close']
testdf['target'] = y_test > 0

In [ ]:
df_imp

In [ ]:
X_te['profits'] = df_profit_te['target']*df_profit_te['Close']

In [ ]:
profits = np.sum(X_te['profits']*X_te['predictions'])

In [ ]:
profits = np.sum(testdf['profits']*testdf['predictions'])

In [ ]:
profits

In [ ]:
#Amount an investore would have lost when investing every time the model is saying the price will go up. False Positive,
np.sum(X_te[((X_te.profits < 0) & (X_te.predictions == 1))]['profits'])

In [ ]:
testing = X_te[(X_te.profits < 0)]

In [ ]:
testing

In [ ]:
#Profits per tweet that was marked with a go ahead
profits/len(testdf[testdf.predictions == 1])

In [ ]:
#Return on Investment
investments_required = testdf[testdf.predictions == 1]
investment = np.sum(investments_required['Close'])
print('Return on Investment: '+str(round((profits/investment)*100,2))+'%')

In [ ]:
#Sort by day
testdf.groupby(['Day'])['predictions'].mean()
Merge

In [ ]:
profits/investment

In [ ]:
testdf[testdf.predictions == 1]*testdf[]

In [ ]:
X_te[X_te.predictions ==1].head(100)

In [ ]:
print('F1 Score Train: '+str(F1(y_tr,y_preds_tr_hard)))
print('F1 Score Validation: '+str(F1(y_te, y_preds_val_hard)))

print('Precision Score Train: '+str(precision(y_tr,y_preds_tr_hard)))
print('Precision Score Validation: '+str(precision(y_te, y_preds_val_hard)))

print('Recall Score Train: '+str(recall(y_tr, y_preds_tr_hard)))
print('Recall Score Validation: '+str(recall( y_te,y_preds_val_hard)))

In [ ]:
print('F1 Score Train: '+str(precision(testdf['target'],predictions_hard)))

print('Precision Score Train: '+str(precision(testdf['target'],predictions_hard)))

print('Recall Score Train: '+str(recall(testdf['target'], predictions_hard)))

In [ ]:
accuracy(testdf['target'], predictions_hard)b

In [ ]:
precision(testdf['target'],predictions_hard)

In [ ]:
len(df_profit_tr[df_profit_tr.target<0])

In [ ]:
precision(y_test, predictions_hard)

In [ ]:
tp = testdf[(testdf.predictions == 1)%(testdf.)]

In [ ]:
#Compute the return
def profit(df, column, classification):
    df_class = df[df[column] == classification]
    

In [ ]:
df_profit_tr['predictions'] = y_preds_tr

In [ ]:
#Stagging
X_tr['target'] = y_tr
X_tr = X_tr.sort_values(by = 'Date')
X_tr['target'] = X_tr['target'].apply(lambda x: convertprices(x))
y_tr = X_tr['target']
X_tr.drop(labels = ['Date','target','Day', 'price_1hour_ahead', 'price_2days_ahead','price_1week_ahead',
'price_2weeks_ahead', 'price_1month_ahead'] , axis = 1, inplace = True)

In [ ]:
y_tr.value_counts()

In [ ]:
start = int(len(X_tr) * 0.4)
targets = 30
accuracy_list = []
precision_list = []
recall_list = []
for i in range(start, len(X_tr), 50):
    print(start)
    startdf = X_tr.iloc[:i]
    targetdf = y_tr.iloc[:i]
    predsdf = X_tr.iloc[i:(i+targets)]
    predsy = y_tr.iloc[i:(i+targets)]
    y_preds_val, y_preds_tr, gbm, df_imp = lightgbm(startdf, predsdf, targetdf, predsy)
    
    #Convert to hard classification
    y_preds_val_hard = y_preds_val > 0.277
    y_preds_tr_hard = y_preds_tr > 0.37
    
    score1 = recall(predsy,y_preds_val_hard)
    score2 = precision(predsy,y_preds_val_hard)
    score3 = accuracy(predsy,y_preds_val_hard)

    recall_list.append(score1)
    precision_list.append(score2)
    accuracy_list.append(score3)
    
    
    print('F1 Score Train: '+str(F1(targetdf,y_preds_tr_hard)))
    print('F1 Score Validation: '+str(F1(predsy, y_preds_val_hard)))
    
    print('Accuracy Score Train: '+str(accuracy(targetdf,y_preds_tr_hard)))
    print('Accuracy Score Validation: '+str(accuracy(predsy, y_preds_val_hard)))

    print('Precision Score Train: '+str(precision(targetdf,y_preds_tr_hard)))
    print('Precision Score Validation: '+str(precision(predsy, y_preds_val_hard)))

    print('Recall Score Train: '+str(recall(targetdf, y_preds_tr_hard)))
    print('Recall Score Validation: '+str(recall(predsy,y_preds_val_hard)))

In [ ]:
y_preds_val

In [ ]:
recall_list

In [ ]:
X_te['target'] = y_te
X_te['target'] = X_te['target'].apply(lambda x: convertprices(x))
y_te_hard = X_te['target']
X_te.drop(labels = ['Date','target','Day', 'price_1hour_ahead', 'price_1week_ahead','price_2days_ahead',
'price_2weeks_ahead', 'price_1month_ahead'] , axis = 1, inplace = True)

In [ ]:
#Test Scores
preds = gbm.predict(X_te, num_iteration=gbm.best_iteration)
preds_hard = preds > 0.277
F1(y_te_hard, preds_hard)

In [ ]:
X_te['predictions'] = preds_hard
X_te['profit_percent'] = y_te
X_te['profits'] = X_te['predictions']*X_te['Close']*X_te['profit_percent']

In [ ]:
profits = np.sum(X_te['profits']*X_te['predictions'])

In [ ]:
profits

In [ ]:
#Amount an investore would have lost when investing every time the model is saying the price will go up. False Positive,
np.sum(X_te[((X_te.profits < 0) & (X_te.predictions == 1))]['profits'])

In [ ]:
#Return on Investment
total_investment = np.sum(X_te[X_te.predictions == 1]['Close'])
ROI = (profits/total_investment)*100

In [ ]:
print('Return on Investment: '+str(ROI)+'%')

In [ ]:
len(X_te[X_te.predictions == 1])

In [ ]:
#Store final model
pkl.dump(X_tr, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/X_train.pkl', 'wb'))
pkl.dump(X_te, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/X_test.pkl', 'wb'))
pkl.dump(y_tr, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/y_train.pkl', 'wb'))
pkl.dump(y_te, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/y_test.pkl', 'wb'))
pkl.dump(preds, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/predictions_lgbm', 'wb'))

In [ ]:
df_imp.to_csv('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Graphs/Feature_Importance.csv')

In [ ]:
#Confusion Matrix
plt.figure(figsize=(10,6))
cm = confusion_matrix(y_te_hard,preds_hard)
cm = cm.astype('float') / cm.sum(axis = 0)[np.newaxis,:]
sns.heatmap(cm,cmap=plt.cm.Blues,annot=True,square=True,
                xticklabels=[0,1],
                yticklabels=[0,1],
                fmt='g')
plt.title('Light GBM Confusion Matrix', size=15)

In [ ]:
accuracy(y_te_hard, preds_hard)

### Regression Model

In [ ]:
pkl.dump(df_model1, open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/model.pkl', 'wb'))

In [ ]:
df_model = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_4/Data/model.pkl', 'rb'))